In [1]:
import sys

BASE_DIR = '../'
sys.path.insert(0, BASE_DIR)

from src import Preprocessor, Logger, KMeansConnector

import configparser
from pyspark.sql import SparkSession

CONFIG_PATH = '../config.ini'

In [2]:
config = configparser.ConfigParser()
config.read(CONFIG_PATH)

['../config.ini']

In [5]:
spark = SparkSession.builder.appName(config['spark']['app_name']) \
    .master(config['spark']['deploy_mode']) \
    .config("spark.driver.host", config['spark']['host'])\
    .config("spark.driver.bindAddress", config['spark']['host']) \
    .config("spark.driver.cores", config['spark']['driver_cores']) \
    .config("spark.executor.cores", config['spark']['executor_cores']) \
    .config("spark.driver.memory", config['spark']['driver_memory']) \
    .config("spark.executor.memory", config['spark']['executor_memory']) \
    .getOrCreate()

24/09/16 18:46:46 WARN Utils: Your hostname, tower resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface enp4s0)
24/09/16 18:46:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 18:46:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
logger = Logger(True).get_logger("logger")
preproc = Preprocessor(logger)

In [10]:
features_names = config['dataset']['features'].split(", ")
output = preproc.load(f"../data/{config['dataset']['name']}", spark, features_names)
scaled_output = preproc.apply_scale(output)

2024-09-16 18:47:13,398 — logger — INFO — START_METHOD: Загрузка датасета в spark
2024-09-16 18:47:13,706 — logger — INFO — END_METHOD: Загрузка датасета в spark
2024-09-16 18:47:13,707 — logger — INFO — START_METHOD: Применение к экземпларам датасета StandardScaler-трансформации
2024-09-16 18:47:13,959 — logger — INFO — END_METHOD: Применение к экземпларам датасета StandardScaler-трансформации


In [8]:
model = KMeansConnector(2, 'scaled_features', logger)

In [11]:
model.fit(scaled_output)

2024-09-16 18:47:16,023 — logger — INFO — START_METHOD: Обучение модели кластеризации
2024-09-16 18:47:18,007 — logger — INFO — END_METHOD: Обучение модели кластеризации


In [12]:
predictions = model.predict(scaled_output)

2024-09-16 18:47:20,972 — logger — INFO — START_METHOD: Получение предсказаний от обученной модели
2024-09-16 18:47:21,006 — logger — INFO — END_METHOD: Получение предсказаний от обученной модели


In [13]:
scores = model.evaluate(predictions)

2024-09-16 18:47:26,282 — logger — INFO — START_METHOD: Оценка качества модели кластеризации
2024-09-16 18:47:26,793 — logger — INFO — END_METHOD: Оценка качества модели кластеризации


In [14]:
scores

0.6272703958697375